## 양방향 RNN을 이용한 품사 태깅
https://wikidocs.net/66747

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext import data
from torchtext import datasets
import time
import random

In [3]:
SEED = 1234
random.seed(SEED)
torch.manual_seed(SEED)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 훈련 데이터
### 필드 정의
사용할 데이터응 총 3개의 열, 즉, 다시 말해 3개의 필드를 가지고 있습니다. 레이블이 총 2개이기 때문인데 이 중 1개만 사용할 것이지만 원활하게 데이터를 불러오기 위해서 일단은 3개 모두 필드를 정의

In [5]:
# 3개의 필드 정의
TEXT = data.Field(lower = True)
UD_TAGS = data.Field(unk_token = None)
PTB_TAGS = data.Field(unk_token = None)

fields = (("text", TEXT), ("udtags", UD_TAGS), ("ptbtags", PTB_TAGS))

### 토치텍스트에서 제공하는 훈련 데이터셋 만들기

In [6]:
train_data, valid_data, test_data = datasets.UDPOS.splits(fields)

downloading en-ud-v2.zip


.data\udpos\en-ud-v2.zip: 100%|██████████| 688k/688k [00:04<00:00, 148kB/s]  


extracting


In [7]:
print(f"훈련 샘플의 개수 : {len(train_data)}")
print(f"검증 샘플의 개수 : {len(valid_data)}")
print(f"테스트 샘플의 개수 : {len(test_data)}")

훈련 샘플의 개수 : 12543
검증 샘플의 개수 : 2002
테스트 샘플의 개수 : 2077


In [8]:
# 훈련 데이터의 3개의 필드 확인
print(train_data.fields)

{'text': <torchtext.data.field.Field object at 0x0000020288CB0148>, 'udtags': <torchtext.data.field.Field object at 0x0000020288CB0108>, 'ptbtags': <torchtext.data.field.Field object at 0x0000020288CB0188>}


In [9]:
# 첫번째 훈련 샘플의 text 필드
print(vars(train_data.examples[0])['text'])

['al', '-', 'zaman', ':', 'american', 'forces', 'killed', 'shaikh', 'abdullah', 'al', '-', 'ani', ',', 'the', 'preacher', 'at', 'the', 'mosque', 'in', 'the', 'town', 'of', 'qaim', ',', 'near', 'the', 'syrian', 'border', '.']


In [10]:
# 첫번째 훈련 샘플의 udtags 필드
print(vars(train_data.examples[0])['udtags'])

['PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'ADJ', 'NOUN', 'VERB', 'PROPN', 'PROPN', 'PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'PROPN', 'PUNCT', 'ADP', 'DET', 'ADJ', 'NOUN', 'PUNCT']


In [11]:
# 첫번째 훈련 샘플의 ptbdtags 필드
print(vars(train_data.examples[0])['ptbtags'])

['NNP', 'HYPH', 'NNP', ':', 'JJ', 'NNS', 'VBD', 'NNP', 'NNP', 'NNP', 'HYPH', 'NNP', ',', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'NNP', ',', 'IN', 'DT', 'JJ', 'NN', '.']


## 단어 집합(Vocabulary) 만들기
단어 집합을 생성 시에 사전 훈련된 워드 임베딩인 GloVe를 사용

In [13]:
# 최소 허용 빈도
MIN_FREQ = 5

# 사전 훈련된 워드 임베딩 GloVe 다운로드
TEXT.build_vocab(train_data, min_freq = MIN_FREQ, vectors = "glove.6B.100d")
UD_TAGS.build_vocab(train_data)
PTB_TAGS.build_vocab(train_data)

.vector_cache\glove.6B.zip: 862MB [20:42, 694kB/s]                                
100%|█████████▉| 399065/400000 [00:18<00:00, 21364.84it/s]

영어에서는 (다른 여러 훈련 데이터에서도) 보통 the가 빈도수가 가장 많습니다. 토치텍스트는 기본적으로 빈도수가 가장 높은 단어부터 작은 숫자를 부여합니다. 물론, <unk>는 0번, <pad>는 1번으로 자동으로 부여되므로 제외입니다.

In [14]:
# 상위 빈도수 20개 단어
print(TEXT.vocab.freqs.most_common(20))

[('the', 9076), ('.', 8640), (',', 7021), ('to', 5137), ('and', 5002), ('a', 3782), ('of', 3622), ('i', 3379), ('in', 3112), ('is', 2239), ('you', 2156), ('that', 2036), ('it', 1850), ('for', 1842), ('-', 1426), ('have', 1359), ('"', 1296), ('on', 1273), ('was', 1244), ('with', 1216)]


In [15]:
# 상위 정수 인덱스 단어 10개 출력
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', '.', ',', 'to', 'and', 'a', 'of', 'i']


In [16]:
# 상위 빈도순으로 udtags 출력
print(UD_TAGS.vocab.freqs.most_common())

[('NOUN', 34781), ('PUNCT', 23679), ('VERB', 23081), ('PRON', 18577), ('ADP', 17638), ('DET', 16285), ('PROPN', 12946), ('ADJ', 12477), ('AUX', 12343), ('ADV', 10548), ('CCONJ', 6707), ('PART', 5567), ('NUM', 3999), ('SCONJ', 3843), ('X', 847), ('INTJ', 688), ('SYM', 599)]


In [17]:
# 상위 정수 인덱스 순으로 출력
print(UD_TAGS.vocab.itos)

['<pad>', 'NOUN', 'PUNCT', 'VERB', 'PRON', 'ADP', 'DET', 'PROPN', 'ADJ', 'AUX', 'ADV', 'CCONJ', 'PART', 'NUM', 'SCONJ', 'X', 'INTJ', 'SYM']


레이블에 속한 단어들의 분포를 출력

In [18]:
def tag_percentage(tag_counts): # 태그 레이블의 분포를 확인하는 함수
    total_count = sum([count for tag, count in tag_counts])
    tag_counts_percentages = [(tag, count, count/total_count) for tag, count in tag_counts]

    return tag_counts_percentages

In [19]:
print("Tag  Occurences Percentage\n")
for tag, count, percent in tag_percentage(UD_TAGS.vocab.freqs.most_common()):
    print(f"{tag}\t{count}\t{percent*100:4.1f}%")

Tag  Occurences Percentage

NOUN	34781	17.0%
PUNCT	23679	11.6%
VERB	23081	11.3%
PRON	18577	 9.1%
ADP	17638	 8.6%
DET	16285	 8.0%
PROPN	12946	 6.3%
ADJ	12477	 6.1%
AUX	12343	 6.0%
ADV	10548	 5.2%
CCONJ	6707	 3.3%
PART	5567	 2.7%
NUM	3999	 2.0%
SCONJ	3843	 1.9%
X	847	 0.4%
INTJ	688	 0.3%
SYM	599	 0.3%


## 데이터로더 만들기

In [20]:
BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [21]:
# 첫번째 미니 배치의 필드를 확인
# 첫번째 미니 배치의 크기는 (시퀀스 길이 × 배치 크기)입니다. batch_first=True를 하지 않았으므로 배치 크기가 두번째 차원이 됩
batch = next(iter(train_iterator))
print(batch)
print(batch.text.shape)
print(batch.text)


[torchtext.data.batch.Batch of size 64 from UDPOS]
	[.text]:[torch.cuda.LongTensor of size 46x64 (GPU 0)]
	[.udtags]:[torch.cuda.LongTensor of size 46x64 (GPU 0)]
	[.ptbtags]:[torch.cuda.LongTensor of size 46x64 (GPU 0)]

## 모델 구현

In [22]:
# 이번 모델에서는 batch_first=True를 사용하지 않으므로 배치 차원이 맨 앞이 아님.
class RNNPOSTagger(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout): 
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers = n_layers, bidirectional = bidirectional)
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)        
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):
        # text = [sent len, batch size]
        embedded = self.dropout(self.embedding(text))

        # embedded = [sent len, batch size, emb dim]
        outputs, (hidden, cell) = self.rnn(embedded)

        # output = [sent len, batch size, hid dim * n directions]
        # hidden/cell = [n layers * n directions, batch size, hid dim]
        predictions = self.fc(self.dropout(outputs))

        # predictions = [sent len, batch size, output dim]
        return predictions

In [24]:
# 실제 클래스로부터 모델 객체로 생성 시에 양방향 여부를 True로 주고, 층의 개수를 2로 합니다.
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 128
OUTPUT_DIM = len(UD_TAGS.vocab)
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = RNNPOSTagger(INPUT_DIM, 
                     EMBEDDING_DIM, 
                     HIDDEN_DIM, 
                     OUTPUT_DIM, 
                     N_LAYERS, 
                     BIDIRECTIONAL, 
                     DROPOUT)

In [25]:
# 파라미터 개수 출력
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,027,510 trainable parameters


## 사전 훈련된 워드 임베딩 사용하기 - Glove
앞서 언급하였던대로, 이번 챕터에서는 사전 훈련된 워드 임베딩인 GloVe를 사용합니다. 이를 위해서는 토치텍스트의 단어 집합 생성 시에 저장해두었던 GloVe 임베딩을 nn.Embedding()에 연결해줄 필요가 있습니다.

In [26]:
# 단어 집합의 단어들에 맵핑된 사전 훈련된 워드 임베딩을 출력합니다.
pretrained_embeddings = TEXT.vocab.vectors
print(pretrained_embeddings.shape)

torch.Size([3921, 100])


단어 집합에 존재하는 총 3,921개의 단어에 대해서 100차원의 벡터가 맵핑되어져 있습니다. 이제 nn.Embedding()에 이를 연결시켜줍니다.

In [27]:
model.embedding.weight.data.copy_(pretrained_embeddings) # 임베딩 벡터값 copy

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.1020,  0.7700,  0.1169,  ..., -0.1416, -0.1932, -0.4225],
        [-0.0263,  0.0179, -0.5016,  ..., -0.8688,  0.9409, -0.2882],
        [ 0.1519,  0.4712,  0.0895,  ..., -0.4702, -0.3127,  0.1078]])

In [28]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
print(UNK_IDX)
print(PAD_IDX)

0
1


In [29]:
# 임의로 0번과 1번 단어에는 0벡터를 만들어줍니다.
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM) # 0번 임베딩 벡터에는 0값을 채운다.
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM) # 1번 임베딩 벡터에는 1값을 채운다.
print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.1020,  0.7700,  0.1169,  ..., -0.1416, -0.1932, -0.4225],
        [-0.0263,  0.0179, -0.5016,  ..., -0.8688,  0.9409, -0.2882],
        [ 0.1519,  0.4712,  0.0895,  ..., -0.4702, -0.3127,  0.1078]])


## 옵티마이저와 비용 함수 구현하기
옵티마이저 설계 전에 레이블 데이터의 패딩 토큰의 인덱스도 확인해봅시다. 0인 것을 확인할 수 있습니다. 이를 하는 이유는 아래 비용 함수를 선택할 때 인자로 주기 위함입니다. 이제 옵티마이저를 설정합니다. 여기서는 Adam을 택했습니다.

In [30]:
TAG_PAD_IDX = UD_TAGS.vocab.stoi[UD_TAGS.pad_token]
print(TAG_PAD_IDX)

0


In [31]:
optimizer = optim.Adam(model.parameters())
# 레이블 데이터의 패딩 토큰은 비용 함수의 연산에 포함시키지도 않도록 레이블 데이터의 패딩 토큰을 무시하라고 기재
criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)
# GPU를 사용 중일때 GPU 연산을 할 수 있도록 지정
model = model.to(device)                                        
criterion = criterion.to(device)

아직 모델은 훈련되지 않은 상태이지만 모델에 입력값을 넣어 출력(예측값)의 크기를 확인해볼까요? 여기서 넣는 입력값은 앞에서 꺼내두었던 첫번째 배치입니다. 46 × 64 × 18은 각각 (첫번째 배치의 시퀀스 길이 × 배치 크기 × 레이블 단어장의 크기)에 해당됩니다. 주의할 점은 현재는 batch_first를 해주지 않아 배치 크기가 맨 앞 차원이 아니라는 점입니다. 또한 46은 첫번째 배치의 시퀀스 길이일뿐, 다른 배치들은 시퀀스 길이가 다를 수 있습니다.

In [32]:
prediction = model(batch.text)
prediction.shape

torch.Size([46, 64, 18])

예측값에 대해서 시퀀스 길이와 배치 길이를 모두 펼쳐주는 작업을 해보겠습니다.

In [33]:
prediction = prediction.view(-1, prediction.shape[-1])
prediction.shape

torch.Size([2944, 18])

In [34]:
batch.udtags.shape

torch.Size([46, 64])

In [35]:
batch.udtags.view(-1).shape

torch.Size([2944])